In [ ]:
import json
import pandas as pd
def readjson(filepath):
    file = open(filepath, 'r', encoding='utf-8')
    dat={}
    for i,line in enumerate(file,start=1):
        dat_new=json.loads(line)
        if i==1:
            dat=dat_new
            for key in dat :
                dat[key]=[dat[key]]
        else:
            for key in dat:
                dat[key].append(dat_new[key])
    file.close()
    data=pd.DataFrame(dat)
    return data
reviews=readjson("../M3Data/review.json")
reviews.head(10)

In [ ]:
user=readjson("../M3Data/user.json")
user.head(10)

In [ ]:
tip=readjson("../M3Data/tip.json")
tip.head(10)

In [ ]:
business=readjson("../M3Data/business.json")

business.head(10)

### Select American Traditional food as target data set.

In [ ]:
business.groupby(['city'])['city'].value_counts().sort_values(ascending=False)[:20]
business_amer=business.loc[(business['categories'].str.contains("American \(Traditional\)",na=False))]
business_amer.head(10)

In [ ]:
business_keep=business_amer.drop(['state','postal_code','hours','address'], axis=1)
business_keep.rename(columns={'review_count':'review_count_business','stars':'stars_business'},inplace=True) 

reviews_keep=reviews.drop(["funny","cool"], axis=1)
reviews_keep.rename(columns={'stars':'stars_review','text':'review_text','date':'date_review'},inplace=True) 

user_keep=user.drop(["name","yelping_since","funny","cool","friends","compliment_more","compliment_profile","compliment_cute","compliment_list","compliment_note","compliment_plain","compliment_cool","compliment_funny","compliment_hot","compliment_writer","compliment_photos"], axis=1)

tip_keep=tip.drop('date', axis=1)
tip_keep.rename(columns={'text':'review_tip','compliment_count':'compliment_count_tip'},inplace=True) 


### merge business, review, user together, select data whose date_review > 2015-01-01 00:00:00.

In [ ]:
business_review=pd.merge(business_keep, reviews_keep, how='inner',on="business_id")
business_review
business_review_user=pd.merge(business_review, user_keep, how='left',on="user_id")
business_review_user
business_review_user=business_review_user[business_review_user["date_review"]>'2015-01-01 00:00:00']
business_review_user

### write data to csv files. 

In [ ]:
business_keep.to_csv(index=True,path_or_buf="./AmericanRestaurant.csv")
business_review_user.to_csv(index=True,path_or_buf="./merged_data.csv")

business_review_user.to_csv(index=True,path_or_buf="C:/Users/wkn98/stat628/628-module-3/merged_data.csv")

In [ ]:
from collections import Counter
category=list(filter(None, business['categories'].values)) 
c = Counter((", ".join(category).split(", ")))
cat=[]
freq=[]
for cat_freq in c.most_common(10):
    cat_1, freq_1 = cat_freq
    cat.append(cat_1)
    freq.append(freq_1)
print(cat, freq)
plt.xticks(rotation=0)
plt.barh(cat,freq)

### Restaurant_category show plot

In [ ]:
business_res=business_keep.loc[(business_keep['categories'].str.contains("Restaurant",na=False))]

category_res=list(filter(None, business_res['categories'].values)) 

def is_remove(x):
    if x=="Restaurants" or x=="Restaurants,Restaurants" or x=="Food":
        return False
    else:
        return True
    
category_res=",".join(category_res).split(", ")
category_res=list(filter(is_remove, category_res)) 



c = Counter(category_res)
cat=[]
freq=[]
for cat_freq in c.most_common(10):
    cat_1, freq_1 = cat_freq
    cat.append(cat_1)
    freq.append(freq_1)
print(cat, freq)
plt.xticks(rotation=0)
plt.barh(cat,freq)
plt.savefig('restaurant_category.pdf')